### This example creates a Satori Dataset, enables it, and then assigns a user to it

In [ ]:
import json
import requests
import datetime

# Authenticate to Satori for a bearer token
# You need to get a service account and key from the Satori management UX
satori_account_id = "YOUR_SATORI_ACCOUNT_ID"
satori_serviceaccount_id = "YOUR_SATORI_SERVICE_ID"
satori_serviceaccount_key = "YOUR_SATORI_SERVICE_KEY"
# API host can be left as-is for all Satori production accounts
satori_apihost = "app.satoricyber.com"

# Various variables for creating a new dataset, enabling it, 
# and then assigning a user to that dataset

# You can search for an exact Satori Datastore ID using its display name
# (or, change this code to use the Datastore ID if you have it already)
desired_satori_datastore = "The Friendly Display Name for a Satori Datastore"

# The desired 1. database name, 2. schema/shard name, and 3. user we want to add
desired_database_name = "postgres"
desired_shard_name = "public"
desired_new_user = "postgres_user:ro"

# The desired duration, measured in hours
desired_access_duration = "5"

# A Satori Security Policy (data filtering, masking) that we will assign to this new user
# Since this is an infrequent or slow-moving item, we have hardwired its ID here :)
# You could also look it up via a different API
desired_security_policy = "SATORI_SECURITY_POLICY_ID"

# Desired Access level, choices are READ_ONLY, READ_WRITE, OWNER
desired_access_level = "READ_ONLY"

### In theory you don't have to change anything below this line

### First, we define several functions and Second, we run them in sequence

In [ ]:
def satori_auth():
    authheaders = {'content-type': 'application/json','accept': 'application/json'}
    url = "https://{}/api/authentication/token".format(satori_apihost)
    payload = """
    {"serviceAccountId": "' + satori_serviceaccount_id + '", "serviceAccountKey": "' + satori_serviceaccount_key + '"}
    """
    try:
        r = requests.post(url, headers=authheaders, data=payload)
        response = r.json()
        satori_token = response["token"]
    except Exception as err:
        print("Bearer Token Failure: :", err)
        print("Exception TYPE:", type(err))
    else:
        return satori_token

### Find our Satori Datastore ID using a search string

In [ ]:
def get_datastore_id(search_string):
    
    get_datastore_url = "https://{}/api/v1/datastore?accountId={}&search={}".format(satori_apihost, 
                                                                                    satori_account_id, 
                                                                                    search_string)
    get_datastore_response = requests.request("GET", get_datastore_url, headers=headers)
    return get_datastore_response.json()['records'][0]['id']

### The next function creates our Satori Dataset - it requires a POST with a body/payload. We have some items hardwired in the payload, and other items are variable-driven

In [ ]:
def create_dataset(datastore_id):
    
    new_dataset_payload = json.dumps({
  "name": desired_database_name + ":" + desired_shard_name ,
  "description": "A new customer and their dataset, added by Satori Rest API",
  "ownersIds": [],
  "includeLocations": [
    {
      "location": {
        "type": "RELATIONAL_LOCATION",
        "db": desired_database_name,
        "schema": desired_shard_name,
        "table": None,
        "column": None,
        "semiPath": None
      },
      "dataStoreId": datastore_id
    }
  ],
  "excludeLocations": []
})
    
    create_dataset_url = "https://{}/api/v1/dataset?accountId={}".format(satori_apihost, 
                                                                         satori_account_id)
    create_dataset_response = requests.request("POST", 
                                               create_dataset_url, 
                                               headers=headers, 
                                               data=new_dataset_payload)
    dataset_policy_id = create_dataset_response.json()['dataPolicyId']
    return dataset_policy_id

### Once our Satori Dataset is created, it also generates a Satori "Policy ID", we now need to Enable it.

In [ ]:
def update_dataset_policy(dataset_policy_id):
    create_dataset_policy_url = "https://{}/api/v1/data-policy/{}/permissions".format(satori_apihost, 
                                                                                      dataset_policy_id)
    create_dataset_policy_body = payload = json.dumps({
    "permissionsEnabled": True
    })
    create_dataset_policy_response = requests.request("PUT", 
                                                      create_dataset_policy_url, 
                                                      headers=headers, 
                                                      data=create_dataset_policy_body)
    return create_dataset_policy_response.text


### And finally, we can add our new user to this Satori Dataset (underneath the hood, we are really associating this user with the Policy ID) - we use another POST with a body/payload. There are some hardwired items here, things like "unused time limit", "days until revocation", etc.

In [ ]:
def add_user_to_dataset(dataset_policy_id):
    add_user_url = "https://{}/api/v1/data-access-rule/instant-access?AccountId={}&parentId={}".format(satori_apihost, 
                                                                                                       satori_account_id, 
                                                                                                       dataset_policy_id)

    add_user_payload = json.dumps(
    {
    "accessLevel": desired_access_level,
    "timeLimit": {
    "shouldExpire": "true",
    "expiration": str(satori_expiration.isoformat())
    },
    "unusedTimeLimit": {
    "unusedDaysUntilRevocation": 1,
    "shouldRevoke": "true"
    },
    "securityPolicyIds": [
    desired_security_policy
    ],
    "identity": {
    "identity": desired_new_user,
    "identityType": "USER"
    }
    })
    
    response = requests.post(add_user_url, headers=headers, data=add_user_payload)
    
    return response.text

### The remainder of this example runs all of the functions above

In [ ]:
# Get our bearer token and generate all future headers
headers = {'Authorization': 'Bearer {}'.format(satori_auth()), 
                                               'Content-Type': 'application/json', 
                                               'Accept': 'application/json'}

In [ ]:
# Convert desired expiration in 'hours' to valid python date
satori_expiration = datetime.datetime.utcnow() + datetime.timedelta(hours=int(float(desired_access_duration))) 

In [ ]:
# Get our Satori Datastore ID by searching for a unique name
found_datastore_id = get_datastore_id(desired_satori_datastore)

# Create a brand new dataset and get its Policy ID
dataset_policy_id = create_dataset(found_datastore_id)

# Set this Policy ID to 'enabled' (secured)
update_dataset_policy(dataset_policy_id)

# Add the new postgres DB username to this dataset, using constants from the top of this example
add_user_to_dataset(dataset_policy_id)
